In [1]:
import os
import time
from configparser import ConfigParser

import pandas as pd
from tqdm import tqdm

from artifact import Artifact, good_to_mona
from build import load_build

proDir = r'D:\yuanshen\artifacts-Filter'

cfg = ConfigParser()
cfg.read(proDir + '/config/screen.ini')

score_threshold = float(cfg.items('screen')[0][1])
rarity_threshold = float(cfg.items('screen')[1][1])

In [2]:
def sort_art(df: pd.DataFrame):
    k = 0  #
    sorted_df = df.iloc[0:1]
    while k != df.shape[0] - 1:
        for t in range(k, df.shape[0]):
            if t < df.shape[0]:
                if df.iloc[t]['adaptScore'] != df.iloc[k]['adaptScore']:
                    p = t - k  # 切片长度
                    slice_sort = df.iloc[k:k + p]
                    slice_sort = slice_sort.sort_values(by='difficulty', ascending=False)
                    sorted_df = pd.concat([sorted_df, slice_sort])
                    k = t  # 下一次起点
            else:
                if df.iloc[t]['adaptScore'] == df.iloc[k]['adaptScore']:
                    p = t - k + 1  # 切片长度
                    slice_sort = df.iloc[k:k + p]
                    slice_sort = slice_sort.sort_values(by='difficulty', ascending=False)
                    sorted_df = pd.concat([sorted_df, slice_sort])
                    k = t  # 最终的终点
        break
    sorted_df = sorted_df[1:]
    return sorted_df


def adapt(art: Artifact, df: pd.DataFrame):
    """对每一个build进行适配性评分"""
    d_temp = {}
    p = 0
    best_score = 0
    for k in range(df.shape[1]):  # 遍历build库
        fitness = 0  # 对该build的适配度
        build_adapt = pd.Series([], dtype='float64')
        fitness += art.main.multiply(df[k]['{}MainWeights'.format(art.position)]).dropna().sum()  # 主属性加权和
        fitness += art.sec.multiply(df[k]['secWeights']).dropna().sum()  # 副属性加权和
        if art.set not in df[k]['sets']:  # 非套装扣0.6
            fitness -= 0.6
        if art.star < 5:  # 非五星扣0.6
            fitness -= 0.6
        adapt_score = fitness / df[k]['best_{}'.format(art.position)]  # 适配分 = 适配度 / 最佳适配度
        # 将build-评分-难度储存到列表中，并以指针为key储存到字典中
        build_adapt['buildName'] = df[k]['buildName']
        build_adapt['adaptScore'] = adapt_score
        build_adapt['difficulty'] = df[k]['{}Difficulty'.format(art.position)]
        d_temp[p] = build_adapt
        p += 1
    art_adapt = pd.DataFrame(d_temp).T  # 字典压缩成df数据表
    best_score = art_adapt['adaptScore'].max()  # 获得最佳适配度
    # 以适配度为第一优先级、毕业难度为第二优先级对df数据表进行排序
    art_adapt = art_adapt.sort_values(by='adaptScore', ascending=False)
    art_adapt = sort_art(art_adapt)
    # 装入圣遗物评分列表前要先筛选掉不合格的适配评分
    art_adapt = art_adapt[art_adapt['adaptScore'] > score_threshold]
    return best_score, art_adapt

In [3]:
build_df = load_build(reload=False)
print('正在分析圣遗物......')
print('只显示适配度大于{}, 或稀有度大于{}的圣遗物.'.format(
    '{:.1%}'.format(score_threshold), '{:.1f}'.format(rarity_threshold)))

art_d = {}
lock_list = []
p_ = 0
artifacts = good_to_mona()


正在分析圣遗物......
只显示适配度大于65.0%, 或稀有度大于8.5的圣遗物.


In [4]:
import pandas as pd
from jsonpath import jsonpath

from dict import attribute_dict, position_dict, set_dict
from rarity import rarity

In [5]:
for idx,artifact in tqdm(artifacts.iterrows()):
    art_dict = artifact.to_dict()
    tar = art_dict['star']
    set = art_dict['setName']
    lock = art_dict['lock']
    set_chs = set_dict[set]["chs"]


1130it [00:00, 7594.41it/s]


In [8]:
r = pd.read_csv('result/result.csv')

In [12]:
for i,s in enumerate(r['result'].to_list()):
    print(f'<============================【{i}】============================>')
    print(s)

<============================【0】============================>
5星 华馆梦醒形骸记 死之羽; 等级:0
主属性为:【攻击力】
副属性为: 元素精通--19; 元素充能效率--6.5%; 防御力%--7.3%; 
该圣遗物最佳评分为:【100.3%】, 稀有度为4.4.
---------build列表---------
对BUILD:[五郎--华馆、绝缘--<充防防>]的适配度为【100.3%】,该部位完美毕业难度为: 4.1
对BUILD:[云堇--华馆--<防防防>]的适配度为【100.3%】,该部位完美毕业难度为: 4.1
对BUILD:[辛焱--辅助--千岩--<防防防>]的适配度为【74.8%】,该部位完美毕业难度为: 4.1
对BUILD:[砂糖--风套--<充精精>]的适配度为【65.5%】,该部位完美毕业难度为: 4.1
对BUILD:[万叶--风套--<精精精>]的适配度为【65.5%】,该部位完美毕业难度为: 4.1
对BUILD:[温迪--辅助--风套--<充风双暴>]的适配度为【65.5%】,该部位完美毕业难度为: 4.1
对BUILD:[温迪--风套--<精精精>]的适配度为【65.5%】,该部位完美毕业难度为: 4.1
<============================【1】============================>
5星 绝缘之旗印 空之杯; 等级:0
主属性为:【防御力%】
副属性为: 生命值--299; 暴击伤害--7.0%; 元素充能效率--6.5%; 
该圣遗物最佳评分为:【100.1%】, 稀有度为5.9.
---------build列表---------
对BUILD:[五郎--华馆、绝缘--<充防防>]的适配度为【100.1%】,该部位完美毕业难度为: 5.6
对BUILD:[辛焱--辅助--千岩--<防防防>]的适配度为【86.7%】,该部位完美毕业难度为: 5.6
对BUILD:[云堇--华馆--<防防防>]的适配度为【86.7%】,该部位完美毕业难度为: 5.6
<============================【2】============================>
5星 海染砗磲 空之杯; 等级:0
主属性为: